# 영업 성공 여부 분류 경진대회
## 팀 : LG영업왕 (김수민, 김준형, 김현동, 정찬혁)

## 1. 데이터 확인

### 필수 라이브러리

In [ ]:
!pip uninstall googletrans -y
!pip install googletrans==3.1.0a0
!pip install catboost

Found existing installation: googletrans 3.1.0a0
Uninstalling googletrans-3.1.0a0:
  Successfully uninstalled googletrans-3.1.0a0
  Using cached googletrans-3.1.0a0-py3-none-any.whl


In [ ]:
import re
import pandas as pd
import numpy as np

from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)

from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score

from googletrans import Translator
translator = Translator()

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

def seed_everything(seed):
    np.random.seed(seed) #numpy를 사용할 경우 고정
seed_everything(42)

### 데이터 셋 읽어오기

In [ ]:
df_train = pd.read_csv("train.csv") # 학습용 데이터
df_test = pd.read_csv("submission.csv") # 테스트 데이터(제출파일의 데이터)

In [ ]:
df_train[:10]

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,idit_strategic_ver,customer_job,lead_desc_length,inquiry_type,product_category,product_subcategory,product_modelname,customer_country.1,customer_position,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,1.00,/Quezon City/Philippines,AS,0.066667,32160,End-Customer,Enterprise,NaN,NaN,NaN,NaN,purchasing,62,Quotation or purchase consultation,multi-split,NaN,NaN,/Quezon City/Philippines,entry level,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Engineering,0,True
1,1.00,/PH-00/Philippines,AS,0.066667,23122,End-Customer,Enterprise,12.0,NaN,NaN,NaN,media and communication,96,Quotation or purchase consultation,multi-split,NaN,NaN,/PH-00/Philippines,ceo/founder,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Advertising,1,True
2,1.00,/Kolkata /India,AS,0.088889,1755,End-Customer,Enterprise,144.0,NaN,NaN,NaN,engineering,56,Product Information,single-split,NaN,NaN,/Kolkata /India,partner,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,Construction,2,True
3,1.00,/Bhubaneswar/India,AS,0.088889,4919,End-Customer,Enterprise,NaN,NaN,NaN,NaN,entrepreneurship,44,Quotation or purchase consultation,vrf,NaN,NaN,/Bhubaneswar/India,ceo/founder,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,IT/Software,3,True
4,1.00,/Hyderabad/India,AS,0.088889,17126,Specifier/ Influencer,Enterprise,NaN,NaN,NaN,NaN,consulting,97,Quotation or purchase consultation,multi-split,NaN,NaN,/Hyderabad/India,partner,LGEIL,less than 3 months,0,0,0.003079,0.026846,corporate / office,NaN,4,True
5,1.00,/Abuja/Nigeria,AS,0.040816,16328,End-Customer,SMB,NaN,NaN,NaN,NaN,program and project management,1114,Quotation or purchase consultation,chiller,NaN,NaN,/Abuja/Nigeria,manager,LGEAF,less than 3 months,1,0,0.003079,0.026846,corporate / office,Engineering,5,True
6,0.75,"/Jeddah, KSA/Saudi Arabia",AS,0.040816,20664,End-Customer,SMB,NaN,NaN,NaN,NaN,engineering,420,Quotation or purchase consultation,single-split,NaN,NaN,"/Jeddah, KSA/Saudi Arabia",manager,LGESJ,NaN,1,0,0.003079,0.026846,corporate / office,Engineering,6,True
7,1.00,/Guwahati/India,AS,0.088889,17983,End-Customer,SMB,NaN,NaN,NaN,NaN,sales,205,Quotation or purchase consultation,vrf,NaN,NaN,/Guwahati/India,partner,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,Manufacturing,7,True
8,0.75,/Cebu city/Philippines,AS,0.066667,30867,Specifier/ Influencer,Enterprise,3.0,NaN,NaN,NaN,other,103,Quotation or purchase consultation,multi-split,NaN,NaN,/Cebu city/Philippines,vice president,LGEPH,less than 3 months,0,0,0.003079,0.026846,corporate / office,Construction,8,True
9,0.75,"/hauz khas,delhi/India",AS,0.088889,6084,End-Customer,SMB,NaN,NaN,NaN,NaN,other,252,Quotation or purchase consultation,vrf,NaN,NaN,"/hauz khas,delhi/India",manager,LGEIL,3 months ~ 6 months,1,0,0.003079,0.026846,corporate / office,NaN,9,True


### 결측치 변수 및 결측률 파악

In [ ]:
mis_val = df_train.isnull().sum()
mis_val_bool = mis_val >= 1
mis_val_df = pd.concat([mis_val, mis_val_bool], axis = 1)
mis_val_df = mis_val_df.rename(columns = {0 : 'mis_val', 1 : 'mis_val_bool'})

mis_val_data = mis_val_df.loc[mis_val_df['mis_val_bool'] == True, :]
mis_val_data['ratio'] = mis_val_data['mis_val'] / 59299
mis_val_data_sorted = mis_val_data.sort_values(by='ratio', ascending=True)
mis_val_data_sorted

<ipython-input-14-2d71e1856ff8>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mis_val_data['ratio'] = mis_val_data['mis_val'] / 59299


,mis_val,mis_val_bool,ratio
inquiry_type,941,True,0.015869
customer_country.1,982,True,0.016560
customer_country,982,True,0.016560
customer_job,18733,True,0.315908
product_category,19374,True,0.326717
expected_timeline,30863,True,0.520464
business_area,40882,True,0.689421
ver_win_rate_x,40882,True,0.689421
customer_type,43961,True,0.741345
ver_win_ratio_per_bu,43995,True,0.741918


In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59299 entries, 0 to 59298
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bant_submit              59299 non-null  float64
 1   customer_country         58317 non-null  object 
 2   business_unit            59299 non-null  object 
 3   com_reg_ver_win_rate     14568 non-null  float64
 4   customer_idx             59299 non-null  int64  
 5   customer_type            15338 non-null  object 
 6   enterprise               59299 non-null  object 
 7   historical_existing_cnt  13756 non-null  float64
 8   id_strategic_ver         3444 non-null   float64
 9   it_strategic_ver         1121 non-null   float64
 10  idit_strategic_ver       4565 non-null   float64
 11  customer_job             40566 non-null  object 
 12  lead_desc_length         59299 non-null  int64  
 13  inquiry_type             58358 non-null  object 
 14  product_category      

## 전처리

### customer_country.1 drop

In [ ]:
# customer_country , customer_country.1의 값이 일치함.
(df_train['customer_country'].fillna(0) == df_train['customer_country.1'].fillna(0)).value_counts()

True    59299
dtype: int64

In [ ]:
pre_train = df_train.drop(labels = ['customer_country.1'],axis=1)
pre_test = df_test.drop(labels = ['customer_country.1','id'],axis=1)

### 수치형/범주형 컬럼 분류

['it_strategic_ver','id_strategic_ver','idit_strategic_ver','lead_owner','ver_cus','ver_pro','customer_idx']
: float -> 범주형 데이터로 변환

In [ ]:
dtypes = pre_train.dtypes

# 각 열의 고유값 개수 확인
unique_value_counts = pre_train.nunique()

# 수치형 변수가 아닌 컬럼을 저장할 리스트 초기화
object_columns = []

# 각 열에 대해 데이터 타입이 'object'이거나 고유값 개수가 일정 이하인 경우를 확인하여 non_numeric_columns 리스트에 추가
for col in pre_train.columns:
    if dtypes[col] == 'object':
        object_columns.append(col)

# 수치형 변수가 아닌 컬럼 출력
print("범주형 컬럼:", object_columns,"\n")
print("수치형 컬럼 :", list(set(pre_train.columns)-set(object_columns)))

범주형 컬럼: ['customer_country', 'business_unit', 'customer_type', 'enterprise', 'customer_job', 'inquiry_type', 'product_category', 'product_subcategory', 'product_modelname', 'customer_position', 'response_corporate', 'expected_timeline', 'business_area', 'business_subarea'] 

수치형 컬럼 : ['lead_desc_length', 'id_strategic_ver', 'historical_existing_cnt', 'ver_cus', 'customer_idx', 'it_strategic_ver', 'ver_win_rate_x', 'ver_pro', 'lead_owner', 'is_converted', 'bant_submit', 'idit_strategic_ver', 'ver_win_ratio_per_bu', 'com_reg_ver_win_rate']


In [ ]:
# 범주형으로 변경
columns_categorized = ['it_strategic_ver','id_strategic_ver','idit_strategic_ver'] + ['lead_owner','ver_cus','ver_pro','customer_idx']

for n in columns_categorized:
    pre_train[n] = pre_train[n].fillna(0)
    pre_train[n] = pre_train[n].astype('int').astype('object')

### 영어로 번역해서 언어 통일

customer_job , inquiry_type , product_category , product_subcategory , product_modelname

#### 번역 test code

In [ ]:
translator = Translator()

def remove_special_characters(text):
    return re.sub(r'\W+', ' ', text)

def is_english(sentence):
    if sentence is np.nan:
        return sentence

    non_alpha_chars = re.sub(r'\W+', '', sentence)
    print(non_alpha_chars)
    non_alpha_chars = re.search(r'[^a-zA-Z]', non_alpha_chars)
    print(non_alpha_chars)

    if non_alpha_chars:
        changed_chars = translator.translate(sentence,dest='en').text
#         return True
    else:
        changed_chars = sentence

    return changed_chars

# 특수 기호가 포함된 문자열 예시
n = np.nan
text_with_special_characters_1 = 'systèmes de débit à réfrigérant variable (drv)'
text_with_special_characters_2 = '醫院電視'
text_with_special_characters_3 = 'refrigerator,built-in/cooking'

# 특수 기호를 제거한 문자열 출력
print(is_english(text_with_special_characters_1))

systèmesdedébitàréfrigérantvariabledrv
<re.Match object; span=(4, 5), match='è'>
variable refrigerant flow (drv) systems


#### 번역 전처리

In [ ]:
def en_dic(df,col):
    unique_value = df[col].unique()
    trans_dic = {}

    for sen in unique_value:
        if sen is np.nan:
            trans_dic[sen] = sen
            continue

        non_alpha_chars = re.sub(r'\W+', '', sen)
        non_alpha_chars = re.search(r'[^a-zA-Z0-9]', non_alpha_chars)

        if non_alpha_chars:
            changed_chars = translator.translate(sen, dest='en').text
            trans_dic[sen] = changed_chars

    return trans_dic

def to_english(df,col):
    # en_dic에 있는 값이면 해당 행 값을 사전의 값으로 교체
    df[col] = df[col].apply(lambda x: dic[x] if x in dic else x)

In [ ]:
need_to_en = ['customer_job' , 'inquiry_type' , 'product_category' , 'product_subcategory' , 'product_modelname']
for n in need_to_en:
    dic = en_dic(pre_train,n)
    to_english(pre_train,n)

for n in need_to_en:
    dic = en_dic(pre_test,n)
    to_english(pre_test,n)

### customer_country : 국가에 해당하는 단어 추출

In [ ]:
# 데이터 통일성을 위해 소문자 변경, 공백 축소
def preprocess_str(col):
    pre_train[col] = pre_train[col].str.lower()
    pre_test[col] = pre_test[col].str.lower()

    mask_train = pre_train[col].notnull()
    mask_test = pre_test[col].notnull()
    pre_train.loc[mask_train, col] = pre_train.loc[mask_train, col].apply(lambda x: x.replace(" ", ""))
    pre_test.loc[mask_test, col] = pre_test.loc[mask_test, col].apply(lambda x: x.replace(" ", ""))

# '/'를 기준으로 customer_country에서 국가에 해당하는 단어 추출
def process_country(country):
    result = np.nan

    if pd.isna(country):
        return np.nan

    parts = [x for x in country.split('/') if x != '']

    if len(parts) >= 1:
        result = parts[-1]
    else:
        return np.nan

    if '@' in str(result):
        return np.nan

    return result

In [ ]:
print("전처리 전 nunique : ")
pre_train['customer_country'].nunique(), pre_test['customer_country'].nunique()

전처리 전 nunique : 


(15399, 2467)

In [ ]:
for n in object_columns:
    preprocess_str(n)

pre_train['customer_country'] = pre_train['customer_country'].apply(process_country)
pre_test['customer_country'] = pre_test['customer_country'].apply(process_country)
pre_train['customer_country'] = pre_train['customer_country'].replace("us", "unitedstates")
pre_test['customer_country'] = pre_test['customer_country'].replace("us", "unitedstates")

In [ ]:
print("전처리 후 nunique : ")
pre_train['customer_country'].nunique(), pre_test['customer_country'].nunique()

전처리 후 nunique : 


(1285, 148)

### customer_job : 같은 직업으로 해석가능한 단어를 묶어서 처리

In [ ]:
# 묶을 수 있는 직업은 처리. 세분화 되어있더라도 모델에게는 아예 다른 직업으로 인식될 것.
def preprocess_job(job):
    if pd.isna(job):
        return np.nan

    if 'direct' in job:
        return 'director'
    elif 'consult' in job:
        return 'consultant'
    elif 'owner' in job:
        return 'owner'
    elif 'market' in job:
        return 'marketor'
    elif 'account' in job:
        return 'account'
    elif 'manage' in job:
        return 'manager'
    elif 'admin' in job:
        return 'administrator'
    elif 'design' in job:
        return 'designer'
    elif 'engineer' in job:
        return 'engineer'
    elif 'manager' in job:
        return 'manager'
    elif 'sales' in job:
        return 'sales'
    elif 'research' in job:
        return 'research'
    elif 'support' in job:
        return 'support'
    elif 'consultant' in job:
        return 'consultant'
    elif 'finance' in job:
        return 'finance'
    elif 'sale' in job:
        return 'sales'
    elif 'educat' or 'teach' in job:
        return 'educator'
    elif 'healthcare'in job:
        return 'healthcare'
    elif 'human' or 'hr posting' in job:
        return 'hr'
    elif 'inform' in job:
        return 'it'
    elif 'military' in job:
        return 'military'
    elif 'comunica' in job:
        return 'comunicator'
    else:
        return job

In [ ]:
# 전처리 전 'admin'을 포함하는 단어로 표현된 값
pre_train['customer_job'][pre_train['customer_job'].fillna('').str.contains('admin')].value_counts()

administrative             3359
administration                8
administrativeassistant       3
admin                         2
pacsadministrator             2
adminassistant                1
platformadministrator         1
imagingadministrator          1
networkadministrator          1
systemsadministrator          1
itadmin                       1
facilityadministrator         1
itadministrator               1
projectadministrator          1
Name: customer_job, dtype: int64

In [ ]:
pre_train['customer_job'].fillna(0)
pre_train['customer_job'] = pre_train['customer_job'].apply(preprocess_job)
pre_test['customer_job'] = pre_test['customer_job'].apply(preprocess_job)

In [ ]:
# 전처리 후
pre_train['customer_job'][pre_train['customer_job'].fillna('').str.contains('admin')].value_counts()

administrator    3383
Name: customer_job, dtype: int64

### other과 유사 단어를 other로 대체
'inquiry_type', 'customer_position', 'customer_type'

In [ ]:
# 'inquiry_type', 'customer_position', 'customer_type' 열에서 특정 문자열을 'other'로 대체
columns_to_replace_other = ['inquiry_type', 'customer_position', 'customer_type']
for col in columns_to_replace_other:
    pre_train[col] = pre_train[col].replace(['other','other_', 'etc.', 'others'], 'other')
    pre_test[col] = pre_test[col].replace(['other','other_', 'etc.', 'others'], 'other')

### product_category : 여러 카테고리를 가진 값을 , 기준으로 분해 -> 증강

In [ ]:
pre_train[pre_train['product_category'] == 'monior/monitortv,tv,commercialtv,pc,refrigerator,solar,rac,washingmachine,mobile,ess,audio/video']

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,idit_strategic_ver,customer_job,lead_desc_length,inquiry_type,product_category,product_subcategory,product_modelname,customer_position,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
22696,1.0,nigeria,solution,NaN,25309,NaN,enterprise,NaN,0,0,0,educator,18,requestforquotationorpurchase,"monior/monitortv,tv,commercialtv,pc,refrigerat...",NaN,NaN,other,lgekr,9months~1year,0,0,0.000543,NaN,specialpurpose,NaN,441,False


In [ ]:
# 'product_category' 열의 각 값을 쉼표로 분할합니다.
pre_train['product_category'] = pre_train['product_category'].str.split(',')

# explode 함수를 사용하여 각 분할된 값을 별도의 행으로 만듭니다.
pre_train = pre_train.explode('product_category')

In [ ]:
pre_train.loc[22696]

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,idit_strategic_ver,customer_job,lead_desc_length,inquiry_type,product_category,product_subcategory,product_modelname,customer_position,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
22696,1.0,nigeria,solution,NaN,25309,NaN,enterprise,NaN,0,0,0,educator,18,requestforquotationorpurchase,monior/monitortv,NaN,NaN,other,lgekr,9months~1year,0,0,0.000543,NaN,specialpurpose,NaN,441,False
22696,1.0,nigeria,solution,NaN,25309,NaN,enterprise,NaN,0,0,0,educator,18,requestforquotationorpurchase,tv,NaN,NaN,other,lgekr,9months~1year,0,0,0.000543,NaN,specialpurpose,NaN,441,False
22696,1.0,nigeria,solution,NaN,25309,NaN,enterprise,NaN,0,0,0,educator,18,requestforquotationorpurchase,commercialtv,NaN,NaN,other,lgekr,9months~1year,0,0,0.000543,NaN,specialpurpose,NaN,441,False
22696,1.0,nigeria,solution,NaN,25309,NaN,enterprise,NaN,0,0,0,educator,18,requestforquotationorpurchase,pc,NaN,NaN,other,lgekr,9months~1year,0,0,0.000543,NaN,specialpurpose,NaN,441,False
22696,1.0,nigeria,solution,NaN,25309,NaN,enterprise,NaN,0,0,0,educator,18,requestforquotationorpurchase,refrigerator,NaN,NaN,other,lgekr,9months~1year,0,0,0.000543,NaN,specialpurpose,NaN,441,False
22696,1.0,nigeria,solution,NaN,25309,NaN,enterprise,NaN,0,0,0,educator,18,requestforquotationorpurchase,solar,NaN,NaN,other,lgekr,9months~1year,0,0,0.000543,NaN,specialpurpose,NaN,441,False
22696,1.0,nigeria,solution,NaN,25309,NaN,enterprise,NaN,0,0,0,educator,18,requestforquotationorpurchase,rac,NaN,NaN,other,lgekr,9months~1year,0,0,0.000543,NaN,specialpurpose,NaN,441,False
22696,1.0,nigeria,solution,NaN,25309,NaN,enterprise,NaN,0,0,0,educator,18,requestforquotationorpurchase,washingmachine,NaN,NaN,other,lgekr,9months~1year,0,0,0.000543,NaN,specialpurpose,NaN,441,False
22696,1.0,nigeria,solution,NaN,25309,NaN,enterprise,NaN,0,0,0,educator,18,requestforquotationorpurchase,mobile,NaN,NaN,other,lgekr,9months~1year,0,0,0.000543,NaN,specialpurpose,NaN,441,False
22696,1.0,nigeria,solution,NaN,25309,NaN,enterprise,NaN,0,0,0,educator,18,requestforquotationorpurchase,ess,NaN,NaN,other,lgekr,9months~1year,0,0,0.000543,NaN,specialpurpose,NaN,441,False


### expected_timeline : 문자열로 표현된 기간을 숫자로 대체

In [ ]:
# 기간이 문자열로 표현됨
pre_train['expected_timeline'].value_counts().iloc[:10]

lessthan3months       17513
3months~6months        5111
morethanayear          3058
9months~1year          1144
6months~9months        1110
lessthan6months         108
etc.                     95
less_than_3_months       76
beingfollowedup          66
beingfollowedup.         24
Name: expected_timeline, dtype: int64

In [ ]:
# 'month', 'year'를 포함하는 문자열인지 판단
def is_period(text):
  if pd.isna(text):
      return np.nan

  for keyword in ['month', 'year']:
    if keyword in text:
      return text
    else:
      continue

  return np.nan

In [ ]:
# 기간으로 표현할 수 있는 데이터만 뽑아냄
period_timeline_train = pre_train['expected_timeline'].apply(is_period).copy()
period_timeline_test = pre_test['expected_timeline'].apply(is_period).copy()

In [ ]:
def extract_numbers(text):
  # 'a', 'one', 'next'을 숫자 1로 대체
  text = re.sub(r'\ba\b', '1', text)
  text = re.sub(r'\bone\b', '1', text)
  text = re.sub(r'\bnext\b', '1', text)

  numbers = re.findall(r'\d+', text)

  cleaned_numbers = []
  for number in numbers:
    # 숫자 뒤에 'month'나 'year'가 있는지 확인하고 number list에 담아줌
    # year이면 12개월이므로 12를 곱해줌
    if re.search(r'\b{}\s*(month)s?\b'.format(number), text):
      cleaned_numbers.append(float(number))
    elif re.search(r'\b{}\s*(year)s?\b'.format(number), text):
      cleaned_numbers.append(float(number)*12)

  return text, cleaned_numbers

extract_numbers('9 months 2 years')

('9 months 2 years', [9.0, 24.0])

In [ ]:
def timeline_to_number(text):
  if pd.isna(text):
        return np.nan

  # 문자열에서 숫자 추출
  _, numbers = extract_numbers(text)
  if len(numbers) == 1:
    num = numbers[0]
    if 'less than' in text:
      return num / 2
    elif 'more than' in text: # 6개월 후로 바꿔줌
      return (6 + num)
    else:
      return num

  elif len(numbers) == 2: # 기간일 때
      num1 = numbers[0]
      num2 = numbers[1]
      return (num1 + num2) / 2

  else:
      return np.nan  # 처리할 수 없는 경우 np.nan 반환

timeline_to_number('9 months 2 years')

16.5

In [ ]:
period_timeline_train = period_timeline_train.apply(timeline_to_number)
period_timeline_test = period_timeline_test.apply(timeline_to_number)

period_mean_train = round(period_timeline_train.mean(), 3)
print(period_mean_train)

5.885


In [ ]:
# object_columns = object_columns + columns_categorized
numerical_features = list(set(pre_train.columns) - set(object_columns))
numerical_features

['lead_desc_length',
 'id_strategic_ver',
 'historical_existing_cnt',
 'expected_timeline_num',
 'ver_cus',
 'customer_idx',
 'it_strategic_ver',
 'ver_win_rate_x',
 'ver_pro',
 'lead_owner',
 'is_converted',
 'bant_submit',
 'idit_strategic_ver',
 'ver_win_ratio_per_bu',
 'com_reg_ver_win_rate']

In [ ]:
# 결측치를 train data의 평균값으로 채워줌
pre_train['expected_timeline_num'] = period_timeline_train.fillna(period_mean_train)
pre_test['expected_timeline_num'] = period_timeline_test.fillna(period_mean_train)
numerical_features.append('expected_timeline_num')

In [ ]:
pre_train['expected_timeline_num'].unique()

array([ 5.885,  4.5  , 10.5  ,  7.5  ,  3.   ,  8.   ])

In [ ]:
pre_test['expected_timeline_num'].unique()

array([ 5.885,  7.5  ,  4.5  , 10.5  ])

### 스케일링

In [ ]:
features = ['com_reg_ver_win_rate','lead_desc_length','ver_win_ratio_per_bu','historical_existing_cnt']

print(pre_train[features].min())
print(pre_train[features].max())

com_reg_ver_win_rate       0.003788
lead_desc_length           1.000000
ver_win_ratio_per_bu       0.011583
historical_existing_cnt    0.000000
dtype: float64
com_reg_ver_win_rate          1.000000
lead_desc_length           1264.000000
ver_win_ratio_per_bu          0.285714
historical_existing_cnt    1394.000000
dtype: float64


In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

pre_train[features] = scaler.fit_transform(pre_train[features]) # fit_transform
pre_test[features] = scaler.transform(pre_test[features]) # transform

In [ ]:
print(pre_train[features].min())
print(pre_train[features].max())

com_reg_ver_win_rate       0.0
lead_desc_length           0.0
ver_win_ratio_per_bu       0.0
historical_existing_cnt    0.0
dtype: float64
com_reg_ver_win_rate       1.0
lead_desc_length           1.0
ver_win_ratio_per_bu       1.0
historical_existing_cnt    1.0
dtype: float64


In [ ]:
pre_train[features].head()

,com_reg_ver_win_rate,lead_desc_length,ver_win_ratio_per_bu,historical_existing_cnt
0,0.063118,0.048298,0.055676,NaN
1,0.063118,0.075218,0.055676,0.008608
2,0.085425,0.043547,0.055676,0.103300
3,0.085425,0.034046,0.055676,NaN
4,0.085425,0.076010,0.055676,NaN


In [ ]:
pre_train.iloc[[1,2]]

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,idit_strategic_ver,customer_job,lead_desc_length,inquiry_type,product_category,product_subcategory,product_modelname,customer_position,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted,expected_timeline_num
1,1.0,philippines,as,0.063118,23122,end-customer,enterprise,0.008608,0,0,0,educator,0.075218,quotationorpurchaseconsultation,multi-split,NaN,NaN,ceo/founder,lgeph,lessthan3months,1,0,0.003079,0.055676,corporate/office,advertising,1,True,5.885
2,1.0,india,as,0.085425,1755,end-customer,enterprise,0.103300,0,0,0,engineer,0.043547,productinformation,single-split,NaN,NaN,partner,lgeil,lessthan3months,1,0,0.003079,0.055676,corporate/office,construction,2,True,5.885


## 3. 모델 학습

### catboost

In [ ]:
# Stratified K-Fold 교차 검증 설정
n_splits = 5  # 원하는 Fold 수
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# 최적의 하이퍼파라미터 설정
best_params = {'depth': 10, 'iterations': 1500, 'learning_rate': 0.01}

# 교차 검증 수행 및 평가 지표 저장
macro_f1_scores = []
models = []
best_f1_score = 0

resetIndex_train = pre_train.reset_index(drop=True)

train_x = resetIndex_train.drop(columns=['is_converted'])
train_y = resetIndex_train['is_converted'].astype(int)
train_x = train_x.fillna(0)

class_weights = compute_class_weight('balanced', classes=[0, 1], y=train_y)

for train_index, valid_index in skf.split(train_x, train_y):
    print(train_index)
    print(valid_index)
    X_train, X_valid = train_x.iloc[train_index], train_x.iloc[valid_index]
    y_train, y_valid = train_y[train_index], train_y[valid_index]

    # CatBoost 모델 생성 및 학습
    model = CatBoostClassifier(loss_function='Logloss', cat_features=np.where(pre_train.dtypes ==object)[0], eval_metric='F1',
                               random_seed=42, verbose=500, **best_params, class_weights = {0:class_weights[0], 1:class_weights[1]})

    model.fit(X_train, y_train, eval_set=[(X_valid,y_valid)])

    # 검증 데이터에 대한 예측 및 평가 (Macro F1 Score)
    valid_predictions = model.predict(X_valid)

    print(valid_predictions)
    print(y_valid[:5])
    macro_f1 = f1_score(y_valid, valid_predictions, average='macro')
    macro_f1_scores.append(macro_f1)
    models.append(model)

    if macro_f1 > best_f1_score:
        best_f1_score = macro_f1
        best_classification_model = model
        y_val = y_valid
        pred_val = valid_predictions

# Macro F1 Score 출력
print("Macro F1 Scores:", macro_f1_scores)


[    2     3     4 ... 59855 59856 59858]
[    0     1     6 ... 59857 59859 59860]
0:	learn: 0.8630886	test: 0.8713049	best: 0.8713049 (0)	total: 354ms	remaining: 8m 51s
500:	learn: 0.9668211	test: 0.9545986	best: 0.9567031 (189)	total: 2m 58s	remaining: 5m 55s
1000:	learn: 0.9783557	test: 0.9501623	best: 0.9567031 (189)	total: 5m 58s	remaining: 2m 58s
1499:	learn: 0.9843714	test: 0.9484081	best: 0.9567031 (189)	total: 9m 7s	remaining: 0us

bestTest = 0.9567031238
bestIteration = 189

Shrink model to first 190 iterations.
[0 1 1 ... 0 0 0]
0     1
1     1
6     1
8     1
16    1
Name: is_converted, dtype: int64
[    0     1     2 ... 59857 59859 59860]
[    3     5    11 ... 59842 59847 59858]
0:	learn: 0.8656501	test: 0.8681595	best: 0.8681595 (0)	total: 336ms	remaining: 8m 23s
500:	learn: 0.9669730	test: 0.9539921	best: 0.9555386 (342)	total: 2m 57s	remaining: 5m 54s
1000:	learn: 0.9782365	test: 0.9525201	best: 0.9555386 (342)	total: 6m 1s	remaining: 3m
1499:	learn: 0.9845013	test: 

In [ ]:
def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[1, 0])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))

In [ ]:
get_clf_eval(y_val, pred_val)

오차행렬:
 [[  928    51]
 [  418 10575]]

정확도: 0.9608
정밀도: 0.6895
재현율: 0.9479
F1: 0.7983


## 4. 제출하기

### 테스트 데이터 예측

In [ ]:
for n in columns_categorized:
    pre_test[n] = pre_test[n].fillna(0)
    pre_test[n] = pre_test[n].astype('int').astype('object')

In [ ]:
# 예측에 필요한 데이터 분리
x_test = pre_test.drop(["is_converted"], axis=1)
x_test.head()

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,idit_strategic_ver,customer_job,lead_desc_length,inquiry_type,product_category,product_subcategory,product_modelname,customer_position,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,expected_timeline_num
0,0.00,brazil,id,0.069725,47466,endcustomer,enterprise,0.038020,0,0,0,consultant,0.001584,NaN,NaN,NaN,NaN,none,lgesp,NaN,1,0,0.001183,0.139558,retail,electronics&telco,278,5.885
1,0.25,unitedstates,it,NaN,5405,endcustomer,smb,NaN,0,0,0,NaN,0.017419,quotationorpurchaseconsultation,NaN,NaN,NaN,none,lgeus,NaN,0,0,0.000013,NaN,transportation,others,437,5.885
2,1.00,u.a.e,id,NaN,13597,specifier/influencer,smb,NaN,0,0,0,educator,0.108472,quotationorpurchaseconsultation,hospitaltv,ut662mseries,50ut662m(mea),manager,lgegf,lessthan3months,0,0,0.000060,0.436158,hospital&healthcare,generalhospital,874,5.885
3,0.50,unitedstates,id,0.115293,17204,NaN,enterprise,NaN,0,0,0,sales,0.001584,quotationorpurchaseconsultation,NaN,digitalsignageorcommercialtvs,NaN,none,lgeus,morethanayear,0,0,0.001183,0.139558,retail,NaN,194,5.885
4,1.00,brazil,id,0.071431,2329,endcustomer,enterprise,0.001435,1,0,1,engineer,0.083927,quotationorpurchaseconsultation,ledsignage,lgledbloc,lsaa012,other,lgesp,lessthan3months,1,1,0.003079,0.193276,corporate/office,engineering,167,5.885


In [ ]:
train_x.head()

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,idit_strategic_ver,customer_job,lead_desc_length,inquiry_type,product_category,product_subcategory,product_modelname,customer_position,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,expected_timeline_num
0,1.0,philippines,as,0.063118,32160,end-customer,enterprise,0.000000,0,0,0,educator,0.048298,quotationorpurchaseconsultation,multi-split,0,0,entrylevel,lgeph,lessthan3months,1,0,0.003079,0.055676,corporate/office,engineering,0,5.885
1,1.0,philippines,as,0.063118,23122,end-customer,enterprise,0.008608,0,0,0,educator,0.075218,quotationorpurchaseconsultation,multi-split,0,0,ceo/founder,lgeph,lessthan3months,1,0,0.003079,0.055676,corporate/office,advertising,1,5.885
2,1.0,india,as,0.085425,1755,end-customer,enterprise,0.103300,0,0,0,engineer,0.043547,productinformation,single-split,0,0,partner,lgeil,lessthan3months,1,0,0.003079,0.055676,corporate/office,construction,2,5.885
3,1.0,india,as,0.085425,4919,end-customer,enterprise,0.000000,0,0,0,educator,0.034046,quotationorpurchaseconsultation,vrf,0,0,ceo/founder,lgeil,lessthan3months,1,0,0.003079,0.055676,corporate/office,it/software,3,5.885
4,1.0,india,as,0.085425,17126,specifier/influencer,enterprise,0.000000,0,0,0,consultant,0.076010,quotationorpurchaseconsultation,multi-split,0,0,partner,lgeil,lessthan3months,0,0,0.003079,0.055676,corporate/office,0,4,5.885


In [ ]:
x_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5271 entries, 0 to 5270
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bant_submit              5271 non-null   float64
 1   customer_country         5239 non-null   object 
 2   business_unit            5271 non-null   object 
 3   com_reg_ver_win_rate     1788 non-null   float64
 4   customer_idx             5271 non-null   object 
 5   customer_type            3814 non-null   object 
 6   enterprise               5271 non-null   object 
 7   historical_existing_cnt  1275 non-null   float64
 8   id_strategic_ver         5271 non-null   object 
 9   it_strategic_ver         5271 non-null   object 
 10  idit_strategic_ver       5271 non-null   object 
 11  customer_job             3832 non-null   object 
 12  lead_desc_length         5271 non-null   float64
 13  inquiry_type             3979 non-null   object 
 14  product_category        

In [ ]:
test_pred = best_classification_model.predict(x_test.fillna(0))
print(test_pred)
sum(int(pred == 1) for pred in test_pred)

[1 1 1 ... 0 1 1]


1643

### 제출 파일 작성

In [ ]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["is_converted"] = test_pred

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)

In [ ]:
best_classification_model.save_model('model.cbm')

In [ ]:
best_classification_model.get_feature_importance(prettified=True)

,Feature Id,Importances
0,customer_idx,18.513968
1,lead_owner,15.856011
2,customer_type,13.008325
3,response_corporate,7.596483
4,product_category,7.125453
5,inquiry_type,4.416452
6,business_unit,3.988520
7,lead_desc_length,3.618843
8,business_area,3.413867
9,customer_position,2.389170


### 모델 저장 및 재검증

In [ ]:
model = CatBoostClassifier()
model = model.load_model('model.cbm')

In [ ]:
for n in columns_categorized:
    pre_test[n] = pre_test[n].fillna(0)
    pre_test[n] = pre_test[n].astype('int').astype('object')

In [ ]:
# 예측에 필요한 데이터 분리
x_test = pre_test.drop(["is_converted"], axis=1)

In [ ]:
test_pred = model.predict(x_test.fillna(0))
print(test_pred)
sum(int(pred == 1) for pred in test_pred)

[1 1 1 ... 0 1 1]


1643